In [1]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense,BatchNormalization,Activation,Dropout,LeakyReLU
from tensorflow.keras.optimizers import SGD,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras.datasets import cifar100



(x_train, Y_train), (x_test, Y_test) = cifar100.load_data()
#x_train = x_train.astype('float32') / 255
#x_test = x_test.astype('float32') / 255

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

aug_data=ImageDataGenerator(rotation_range=20,horizontal_flip=True,width_shift_range=0.1,shear_range = 0.2,height_shift_range=0.1,zoom_range=0.2,brightness_range = (0.5, 1.5))
aug_data.fit(x_train)

169009152/169001437 [==============================] - 4s 0us/step


In [2]:

#import tensorflow as 
model = Sequential()

# Creating first block- (2 Convolution + 1 Max pool)
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same', input_shape= (32, 32, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Creating second block- (2 Convolution + 1 Max pool)
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))


# Creating third block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Creating fourth block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(2,2)))

# Creating fifth block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Flattening the pooled image pixels
model.add(Flatten())

# Creating 2 Dense Layers
model.add(Dense(units= 512))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(units= 100, activation='softmax'))
adam=Adam(learning_rate=0.0001,clipnorm=1,name='adam')


model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

checkpoint = ModelCheckpoint("VGG_Adam_no_reg_weights_amit.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto',restore_best_weights=True)


In [3]:
model.fit(aug_data.flow(x_train, y_train, batch_size=128), batch_size=128, epochs=100, verbose=1, validation_data=(x_test, y_test),callbacks=[checkpoint,early])


Epoch 1/100
391/391 [==============================] - ETA: 0s - loss: 3.9238 - accuracy: 0.1094
Epoch 00001: val_accuracy improved from -inf to 0.21810, saving model to VGG_Adam_no_reg_weights_amit.hdf5
391/391 [==============================] - 335s 858ms/step - loss: 3.9238 - accuracy: 0.1094 - val_loss: 3.2499 - val_accuracy: 0.2181
Epoch 2/100
391/391 [==============================] - ETA: 0s - loss: 3.1673 - accuracy: 0.2418
Epoch 00002: val_accuracy improved from 0.21810 to 0.31050, saving model to VGG_Adam_no_reg_weights_amit.hdf5
391/391 [==============================] - 333s 852ms/step - loss: 3.1673 - accuracy: 0.2418 - val_loss: 2.7756 - val_accuracy: 0.3105
Epoch 3/100
391/391 [==============================] - ETA: 0s - loss: 2.7751 - accuracy: 0.3157
Epoch 00003: val_accuracy improved from 0.31050 to 0.36800, saving model to VGG_Adam_no_reg_weights_amit.hdf5
391/391 [==============================] - 333s 852ms/step - loss: 2.7751 - accuracy: 0.3157 - val_loss: 2.4980 

In [4]:
y_pred=model.predict_classes(x_test)
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("Accuracy is {}".format(accuracy_score(Y_test,y_pred)))
print("Precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("Recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
              precision    recall  f1-score   support

           0       0.81      0.81      0.81       100
           1       0.69      0.83      0.75       100
           2       0.57      0.56      0.57       100
           3       0.48      0.42      0.45       100
           4       0.43      0.47      0.45       100
           5       0.65      0.67      0.66       100
           6       0.74      0.70      0.72       100
           7       0.54      0.71      0.61       100
           8       0.67      0.78      0.72       100
           9       0.70      0.67      0.68       100
          10       0.48      0.44      0.46       100
          11     

In [ ]:
YY=model.predict_classes(x_train)
print("Accuracy is {}".format(accuracy_score(Y_train,YY)))

In [ ]:
model.summary()

In [1]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense,BatchNormalization,Activation,Dropout,LeakyReLU
from tensorflow.keras.optimizers import SGD,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras.datasets import cifar100



(x_train, Y_train), (x_test, Y_test) = cifar100.load_data()
#x_train = x_train.astype('float32') / 255
#x_test = x_test.astype('float32') / 255

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

aug_data=ImageDataGenerator(rotation_range=20,horizontal_flip=True,width_shift_range=0.1,shear_range = 0.2,height_shift_range=0.1,zoom_range=0.2,brightness_range = (0.5, 1.5))
aug_data.fit(x_train)


#import tensorflow as 
model = Sequential()

# Creating first block- (2 Convolution + 1 Max pool)
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same', input_shape= (32, 32, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Creating second block- (2 Convolution + 1 Max pool)
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))


# Creating third block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Creating fourth block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(2,2)))

# Creating fifth block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Flattening the pooled image pixels
model.add(Flatten())

# Creating 2 Dense Layers
model.add(Dense(units= 512))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(units= 100, activation='softmax'))


#adam=Adam(learning_rate=0.0001,clipnorm=1,name='adam')

model.load_weights('../weights/VGG_Adam_no_reg_weights_amit.hdf5')

y_pred=model.predict_classes(x_test)
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("Accuracy is {}".format(accuracy_score(Y_test,y_pred)))
print("Precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("Recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))




Using TensorFlow backend.




              precision    recall  f1-score   support

           0       0.81      0.81      0.81       100
           1       0.69      0.83      0.75       100
           2       0.57      0.56      0.57       100
           3       0.48      0.42      0.45       100
           4       0.43      0.47      0.45       100
           5       0.65      0.67      0.66       100
           6       0.74      0.70      0.72       100
           7       0.54      0.71      0.61       100
           8       0.67      0.78      0.72       100
           9       0.70      0.67      0.68       100
          10       0.48      0.44      0.46       100
          11       0.40      0.61      0.48       100
          12       0.62      0.77      0.69       100
          13       0.60      0.63      0.61       100
          14       0.61      0.65      0.63       100
          15       0.55      0.63      0.59       100
          16       0.68      0.64      0.66       100
          17       0.78  